In [ ]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /private/var/folders/46/8yh56x491r1cj78tqrzzgzqc0000gn/T/pip-install-0g19sjco/unsloth_08f87026fa8f45a7aa92051a3c0c1559
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /private/var/folders/46/8yh56x491r1cj78tqrzzgzqc0000gn/T/pip-install-0g19sjco/unsloth_08f87026fa8f45a7aa92051a3c0c1559


In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

ModuleNotFoundError: No module named 'unsloth'

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
text = "You will only answer in nepali, here is the question: तिम्रो नाम के हो ?"
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(text, return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1000, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>You will only answer in nepali, here is the question: तिम्रो नाम के हो ?\n\nAnswer:\n\nStep 1/2\nFirst, we need to understand the question in Nepali. The question is asking for the name of the person who is asking the question. So, the question is: "Who are you?"\n\nStep 2/2\nNow, we can answer the question in Nepali: "मैं तिम्रो नाम के हो।" (I am yours, my name is yours.)<eos>']

<a name="Data"></a>

### Data Prep

We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Context:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Context:
{}

### Input:
{}

### Response:
{}"""

inputs = tokenizer(
[
    prompt.format(
        "Given the context, you have to reply it to the user in NEPALI letters. Don't hallucinate or repeat same, just give the exact answer.", # instruction
        """नेपाल दक्षिण एसियाको एउटा सानो तर प्राकृतिक र सांस्कृतिक रूपमा समृद्ध देश हो। यो देश विश्वको सर्वोच्च शिखर सगरमाथाबाट लिएर तराईका समथर फाँटसम्म फैलिएको छ। यसको भौगोलिक विविधता संसारमा अरू कुनै देशसँग तुलना गर्न सकिँदैन। यहाँ तीन भौगोलिक क्षेत्रहरू छन्—हिमाल, पहाड, र तराई। हिमाल क्षेत्रमा धेरै प्रसिद्ध हिमालहरू छन्, जस्तै सगरमाथा, अन्नपूर्ण, र मनास्लु। यी हिमालहरूले नेपाललाई ट्रेकिङ र पर्वतारोहणको गन्तव्यका रूपमा परिचित बनाएका छन्।
        नेपालको सांस्कृतिक विविधता पनि यहाँको प्रमुख पहिचान हो। नेपालमा १२५ भन्दा बढी जातजाति र १२३ भाषाभाषी समुदाय बसोबास गर्छन्। यी सबै जातजातिका आफ्नै परम्परा, भेषभूषा, र चाडपर्व छन्। दशैं र तिहारजस्ता मुख्य चाडपर्व सबै जातीय समुदायले मनाउने भए पनि, किराँतहरूको उभौली र उधौली, तामाङहरूको ल्होसर, र मधेसी समुदायको छठजस्ता चाडपर्वले सांस्कृतिक विविधतालाई झल्काउँछन्। धार्मिक रूपमा पनि नेपाल बहुधार्मिक देश हो। यहाँ हिन्दू, बौद्ध, इस्लाम, क्रिश्चियनिटी, र किराँत धर्मावलम्बीहरू शान्तिपूर्ण सह-अस्तित्वमा बस्छन्। पशुपतिनाथ मन्दिर, लुम्बिनी, स्वयम्भूनाथ, र जनकपुरधाम नेपालका प्रसिद्ध धार्मिक स्थल हुन्।
        इतिहासको कुरा गर्दा, नेपाल एक स्वतन्त्र राष्ट्र हो, जुन कहिल्यै पनि कुनै विदेशी उपनिवेश बनेको थिएन। प्राचीन समयमा नेपाललाई "नेवार" वा "नेपाल मण्डल" भनेर चिनिन्थ्यो। गोर्खा राज्यका राजा पृथ्वीनारायण शाहले नेपाललाई एकीकृत गरी आधुनिक नेपालको सुरुवात गरेका थिए। नेपालले २००७ सालमा प्रजातन्त्रको सुरुवात गर्‍यो। त्यसपछि २०६२/६३ को आन्दोलनबाट नेपाल लोकतान्त्रिक गणतन्त्र बनेको हो। हाल नेपालमा सङ्घीय संरचनामा सातवटा प्रदेशहरू छन्।
        शिक्षाको क्षेत्रमा पनि नेपालले पछिल्लो समयमा राम्रो प्रगति गरिरहेको छ। प्राथमिक शिक्षामा उल्लेखनीय सुधार भए पनि उच्च शिक्षामा अझै सुधार गर्नुपर्ने आवश्यकता छ। काठमाडौं विश्वविद्यालय, त्रिभुवन विश्वविद्यालय, र पुर्वाञ्चल विश्वविद्यालयले नेपाली विद्यार्थीहरूलाई उच्च शिक्षाको अवसर प्रदान गरिरहेका छन्। यद्यपि, विज्ञान, प्रविधि, र अनुसन्धानमा नेपाल अझै धेरै पछाडि छ।
        नेपालमा धेरै सामाजिक समस्याहरू पनि छन्। बेरोजगारी, अशिक्षा, गरिबी, र लैङ्गिक असमानता यहाँका प्रमुख समस्या हुन्। विशेषगरी, दुर्गम क्षेत्रका मानिसहरूले स्वास्थ्य सेवा र शिक्षामा अझै पनि पर्याप्त पहुँच पाउन सकेका छैनन्। जलवायु परिवर्तनले नेपालको प्राकृतिक स्रोतमा नकारात्मक असर पारेको छ। हिमनदीहरू पग्लँदै जानु र अनियन्त्रित वन कटानीले पर्यावरणीय असन्तुलन निम्त्याइरहेको छ।
        यद्यपि, प्रविधिको विकासले नेपालमा धेरै सकारात्मक परिवर्तनहरू ल्याएको छ। इन्टरनेट र मोबाइल प्रविधिले नेपाली जनताको जीवनलाई सहज बनाएको छ। मोबाइल बैंकिङ, ई-कमर्स, र डिजिटल शिक्षा अहिले लोकप्रिय बन्दै गइरहेका छन्। नेपालले सूचना प्रविधिमा प्रगति गरिरहेको भए पनि, डिजिटल विभाजनको समस्या अझै बाँकी छ। ग्रामीण क्षेत्रका मानिसहरूले प्रविधिको फाइदा लिन सकेका छैनन्।
        यसरी, नेपाल प्राकृतिक, सांस्कृतिक, र ऐतिहासिक रूपमा धनी भए पनि, यहाँका विभिन्न सामाजिक समस्या समाधान गर्न अझै धेरै प्रयास गर्नुपर्ने आवश्यकता छ। सरकारी तथा गैरसरकारी क्षेत्रहरूले मिलेर नेपाललाई आर्थिक, सामाजिक, र प्रविधिको क्षेत्रमा अगाडि बढाउन योगदान पुर्‍याउनु महत्त्वपूर्ण छ।""", # context
        """नेपाललाई भौगोलिक रूपमा कुन तीन क्षेत्रमा विभाजन गरिएको छ?""", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 150, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven the context, you have to reply it to the user in NEPALI letters. Don\'t hallucinate or repeat same, just give the exact answer.\n\n### Context:\nनेपाल दक्षिण एसियाको एउटा सानो तर प्राकृतिक र सांस्कृतिक रूपमा समृद्ध देश हो। यो देश विश्वको सर्वोच्च शिखर सगरमाथाबाट लिएर तराईका समथर फाँटसम्म फैलिएको छ। यसको भौगोलिक विविधता संसारमा अरू कुनै देशसँग तुलना गर्न सकिँदैन। यहाँ तीन भौगोलिक क्षेत्रहरू छन्—हिमाल, पहाड, र तराई। हिमाल क्षेत्रमा धेरै प्रसिद्ध हिमालहरू छन्, जस्तै सगरमाथा, अन्नपूर्ण, र मनास्लु। यी हिमालहरूले नेपाललाई ट्रेकिङ र पर्वतारोहणको गन्तव्यका रूपमा परिचित बनाएका छन्।\n        नेपालको सांस्कृतिक विविधता पनि यहाँको प्रमुख पहिचान हो। नेपालमा १२५ भन्दा बढी जातजाति र १२३ भाषाभाषी समुदाय बसोबास गर्छन्। यी सबै जातजातिका आफ्नै परम्परा, भेषभूषा, र चाडपर्व छन्। दशैं र तिहारजस्ता मुख्य चाडपर्व सबै जाती

<a name="Train"></a>

### Train the model

Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 240,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.428 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 240
 "-____-"     Number of trainable parameters = 19,611,648
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
1,1.892100
2,2.470700
3,2.027200
4,2.318000
5,2.093700
6,2.035600
7,1.484900
8,1.758400
9,1.598500
10,1.665700


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

537.0524 seconds used for training.
8.95 minutes used for training.
Peak reserved memory = 5.689 GB.
Peak reserved memory for training = 3.261 GB.
Peak reserved memory % of max memory = 38.575 %.
Peak reserved memory for training % of max memory = 22.111 %.


<a name="Inference"></a>

### Inference

Let's run the model! You can change the instruction and input - leave the output blank!


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Context:
{}

### Input:
{}

### Response:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Given the context, you have to reply it to the user in NEPALI letters.", # instruction
        "मेरो नाम राम हो", # context
        "तिम्रो नाम के हो ?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 200, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGiven the context, you have to reply it to the user in NEPALI letters.\n\n### Context:\nमेरो नाम राम हो\n\n### Input:\nतिम्रो नाम के हो ?\n\n### Response:\nYour name is Ram.<eos>']

You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 1000)

<bos>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
The next term in the Fibonacci sequence fter the given term 8 is 13.<eos>


<a name="Save"></a>

### Saving, loading finetuned models

To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!


In [ ]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:


In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

IndexError: Replacement index 3 out of range for positional args tuple

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.


In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.


In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion

To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):

- `q8_0` - Fast conversion. High resource use, but generally acceptable.
- `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
- `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.


In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).


And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:

1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>
